In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
import shapefile
from polylabel import polylabel

In [2]:
%%time
coor_df = pd.read_csv('./Prepped Data/연속지적도출처_지번별좌표_baseline_20211222.csv')
print(coor_df.shape)
coor_df.head()

(922318, 7)
Wall time: 984 ms


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표
0,강남구,논현동,67-26 도,67-26,도,202648.630547,446277.538516
1,강남구,논현동,67-27 대,67-27,대,202676.093470,446297.563160
2,강남구,논현동,67-28 대,67-28,대,202584.375000,446275.930000
3,강남구,논현동,68 대,68,대,202695.885000,446317.525000
4,강남구,논현동,68-1 대,68-1,대,202707.844582,446324.121631


In [5]:
def contains_a_letter(x):
    #x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    for i in range(10):
        x = x.replace(str(i), '')
    
    x = x.replace('-', '')
    
    if x == '':
        return 0
    else:
        return 1

In [6]:
coor_df['글자있음'] = coor_df['지번'].apply(contains_a_letter)

In [7]:
coor_df.head()

,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음
0,강남구,논현동,67-26 도,67-26,도,202648.630547,446277.538516,0
1,강남구,논현동,67-27 대,67-27,대,202676.093470,446297.563160,0
2,강남구,논현동,67-28 대,67-28,대,202584.375000,446275.930000,0
3,강남구,논현동,68 대,68,대,202695.885000,446317.525000,0
4,강남구,논현동,68-1 대,68-1,대,202707.844582,446324.121631,0


In [8]:
letter_df = coor_df[coor_df['글자있음'] == 1]
print(letter_df.shape)
letter_df.head()

(16952, 8)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음
318,강남구,일원동,산63-39 임,산63-39,임,207139.311391,441737.168547,1
442,강남구,개포동,산63-5임,산63-5,임,206367.663594,442281.921656,1
443,강남구,개포동,산64-5임,산64-5,임,206295.416734,442295.344234,1
502,강남구,자곡동,산39-4임,산39-4,임,208367.011547,441593.207922,1
667,강남구,개포동,산53-38 임,산53-38,임,205930.885391,441484.142484,1


In [10]:
def find_letters(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    for i in range(10):
        x = x.replace(str(i), '')
    
    x = x.replace('-', '')
    
    if x == '':
        return '글자없음'
    else:
        return x

In [11]:
coor_df['글자'] = coor_df['지번'].apply(find_letters)

In [12]:
coor_df.head()

,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자
0,강남구,논현동,67-26 도,67-26,도,202648.630547,446277.538516,0,글자없음
1,강남구,논현동,67-27 대,67-27,대,202676.093470,446297.563160,0,글자없음
2,강남구,논현동,67-28 대,67-28,대,202584.375000,446275.930000,0,글자없음
3,강남구,논현동,68 대,68,대,202695.885000,446317.525000,0,글자없음
4,강남구,논현동,68-1 대,68-1,대,202707.844582,446324.121631,0,글자없음


In [13]:
has_letters_df = coor_df[coor_df['글자'] != '글자없음']
print(has_letters_df.shape)
has_letters_df.head()

(16952, 9)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자
318,강남구,일원동,산63-39 임,산63-39,임,207139.311391,441737.168547,1,산
442,강남구,개포동,산63-5임,산63-5,임,206367.663594,442281.921656,1,산
443,강남구,개포동,산64-5임,산64-5,임,206295.416734,442295.344234,1,산
502,강남구,자곡동,산39-4임,산39-4,임,208367.011547,441593.207922,1,산
667,강남구,개포동,산53-38 임,산53-38,임,205930.885391,441484.142484,1,산


In [14]:
has_letters_df['글자'].value_counts()

산    16951
도        1
Name: 글자, dtype: int64

In [16]:
def remove_letters(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    y = x
    
    for i in range(10):
        x = x.replace(str(i), '')
    
    x = x.replace('-', '')
    
    if x == '':
        return y
    else:
        for v in x:
            y = y.replace(v, '')
        
        return y

In [17]:
coor_df['지번만'] = coor_df['지번'].apply(remove_letters)

In [18]:
coor_df.head()

,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만
0,강남구,논현동,67-26 도,67-26,도,202648.630547,446277.538516,0,글자없음,67-26
1,강남구,논현동,67-27 대,67-27,대,202676.093470,446297.563160,0,글자없음,67-27
2,강남구,논현동,67-28 대,67-28,대,202584.375000,446275.930000,0,글자없음,67-28
3,강남구,논현동,68 대,68,대,202695.885000,446317.525000,0,글자없음,68
4,강남구,논현동,68-1 대,68-1,대,202707.844582,446324.121631,0,글자없음,68-1


In [19]:
has_letters_df2 = coor_df[coor_df['글자'] != '글자없음']
print(has_letters_df2.shape)
has_letters_df2.head()

(16952, 10)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만
318,강남구,일원동,산63-39 임,산63-39,임,207139.311391,441737.168547,1,산,63-39
442,강남구,개포동,산63-5임,산63-5,임,206367.663594,442281.921656,1,산,63-5
443,강남구,개포동,산64-5임,산64-5,임,206295.416734,442295.344234,1,산,64-5
502,강남구,자곡동,산39-4임,산39-4,임,208367.011547,441593.207922,1,산,39-4
667,강남구,개포동,산53-38 임,산53-38,임,205930.885391,441484.142484,1,산,53-38


In [22]:
coor_df['본번'] = coor_df['지번만'].apply(lambda x: int(x.split('-')[0]))
coor_df['부번'] = coor_df['지번만'].apply(lambda x: int(x.split('-')[1]) if len(x.split('-')) > 1 else 0)

In [23]:
zero_df = coor_df[coor_df['부번'] == 0]
print(zero_df.shape)
zero_df.head()

(54950, 12)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만,본번,부번
3,강남구,논현동,68 대,68,대,202695.885000,446317.525000,0,글자없음,68,68,0
6,강남구,삼성동,171 대,171,대,205789.770625,445761.130625,0,글자없음,171,171,0
9,강남구,신사동,653 대,653,대,203229.315000,446930.395000,0,글자없음,653,653,0
38,강남구,신사동,609 대,609,대,202536.517188,447118.887187,0,글자없음,609,609,0
45,강남구,신사동,610 대,610,대,202500.490625,447159.779375,0,글자없음,610,610,0


In [24]:
coor_df = coor_df.sort_values(['시군구명', '법정동명', '본번', '부번']).reset_index(drop=True)
print(coor_df.shape)
coor_df.head()

(922318, 12)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만,본번,부번
0,강남구,개포동,0-1 가,0-1,가,206331.014484,442028.661953,0,글자없음,0-1,0,1
1,강남구,개포동,0-2 가,0-2,가,206254.177141,441986.263641,0,글자없음,0-2,0,2
2,강남구,개포동,0-3 가,0-3,가,206174.409344,441932.356344,0,글자없음,0-3,0,3
3,강남구,개포동,0-4 가,0-4,가,206631.548237,442577.897175,0,글자없음,0-4,0,4
4,강남구,개포동,0-5 가,0-5,가,205243.406000,442547.328000,0,글자없음,0-5,0,5


In [25]:
%%time
coor_df.to_csv('./Prepped Data/연속지적도출처_지번별좌표_본번부번정리_20211222.csv', index=False)

Wall time: 3.02 s
